In [1]:
"""
This script reads two CSV files containing data previously scraped with the "Data scraper Inderes" code, merges them into a single DataFrame, and preprocesses the data for analysis.

1. Reading pickle File:
   - The reads pickel file saved earlier in the process from running ths scraper.
   
3. Copying DataFrame:
   - To ensure data integrity, the script creates a copy of the merged DataFrame for further processing.
   
4. Preprocessing Data:
   - The script converts string representations of dictionaries in the 'Comments Details' column to actual dictionaries using the 'ast' module.
   - It explodes the 'Comments Details' column to create a new row for each comment, and extracts specific details from the dictionaries.
   - Comments are aggregated while merging with a unique DataFrame.

5. Handling Date and Timestamps:
   - Finnish dates and timestamps are converted into standard formats (with four-digit years) using predefined mappings.
   - Time-related columns are converted to datetime objects for ease of analysis.

6. Identifying Company Mentions:
   - The script identifies company mentions in comments using pre-defined patterns and Named Entity Recognition (NER) classification.
   - It processes comments to extract validated matches of company mentions.

7. Converting Numeric Representations:
   - Mixed numeric representations are converted to integers for consistency and ease of analysis.
   
8. Processnng the comment data and calculating sentiments for comments.
   - Preprocessing comments and using text classification model to calculate sentiments by using fergusq/finbert-finnsentiment- model.

9. Saving Preprocessed Data:
   - The preprocessed DataFrame is saved to a pickle file for future use.

10. Execution Time:
   - The script calculates the elapsed time for execution and prints it for monitoring purposes.

Note:
- Ensure that all necessary libraries are installed before running the script.
- Adjust the file paths and other parameters as needed.
"""

import time
import re
import pandas as pd
import ast
from datetime import datetime, timedelta
from transformers import pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import os




# Define the file path for the dataset
file_path1 = 'forum_data_scraped2'


# Start time to measure execution duration
start_time = time.time()

def read_pickle(file_path):
    """
    Read a pickle file.

    Parameters:
        file_path (str): The file path of the pickle file to be read.

    Returns:
        pd.DataFrame: A DataFrame containing the data read from the pickle file.
    """
    try:
        df = pd.read_pickle(file_path)
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        df = None
    except Exception as e:
        print(f"Error reading the pickle file {file_path}:", e)
        df = None
    return df

# Read the dataset from the pickle file
data_df = read_pickle(file_path1)

# Check if the DataFrame is loaded successfully
if data_df is not None:
    # Display the first few rows to inspect the data
    print("First few rows of DataFrame:")
    print(data_df.head())

    # You can now perform further processing as needed
    # Example: Renaming columns for clarity
    data_df.rename(columns={'Likes': 'Post Likes'}, inplace=True)

    # Example: Convert string representations of dictionaries in 'Comments Details' column to actual dictionaries
    data_df['Comments Details'] = data_df['Comments Details'].apply(lambda x: eval(x) if isinstance(x, str) else x)

    # Example: Convert date columns to datetime
    date_columns = ['Date', 'Time']  # Replace with actual column names in your DataFrame
    for col in date_columns:
        if col in data_df.columns:
            data_df[col] = pd.to_datetime(data_df[col], errors='coerce')

    # Additional processing can be added here
else:
    print("Data could not be loaded properly.")


First few rows of DataFrame:
                                                 URL   Created At  \
0  https://keskustelut.inderes.fi/t/onnistumiset-...   touko 2018   
1  https://keskustelut.inderes.fi/t/macys-inc-tav...  maalis 2020   
2  https://keskustelut.inderes.fi/t/suomalaisten-...    syys 2018   
3  https://keskustelut.inderes.fi/t/nattopharma-j...   joulu 2020   
4  https://keskustelut.inderes.fi/t/lindex-group-...   heinä 2021   

    Last Reply  Visits Replies Users   Likes  \
0   joulu 2023  19,0 k      71    48     545   
1    4. maalis  21,3 k     139    24     457   
2    loka 2018   6,3 k      23     9      66   
3  maalis 2021  15,4 k     138    35     546   
4         2 pv   491 k   2,5 k   260  30,8 k   

                                    Comments Details  
0  [{'comment': 'Tässä ketjussa voi kehua omia on...  
1  [{'comment': 'Macy’s Inc on perinteinen Yhdysv...  
2  [{'comment': 'Harrastuksen vuoksi haluaisin py...  
3  [{'comment': 'NattoPharma Corporate Video', 

In [ ]:
data_df.info()

In [2]:
# Rename the 'Likes' column to 'Post Likes'
data_df.rename(columns={'Likes': 'Post Likes'}, inplace=True)

In [3]:
# Remove trailing slash followed by digits from URLs
data_df['URL'] = data_df['URL'].str.replace(r'/\d+$', '', regex=True)

# Display the modified URLs
print(data_df['URL'])

0      https://keskustelut.inderes.fi/t/onnistumiset-...
1      https://keskustelut.inderes.fi/t/macys-inc-tav...
2      https://keskustelut.inderes.fi/t/suomalaisten-...
3      https://keskustelut.inderes.fi/t/nattopharma-j...
4      https://keskustelut.inderes.fi/t/lindex-group-...
                             ...                        
784    https://keskustelut.inderes.fi/t/dermtech-ihom...
785    https://keskustelut.inderes.fi/t/supermicro-sm...
786    https://keskustelut.inderes.fi/t/terrafame-pal...
787    https://keskustelut.inderes.fi/t/embracer-grou...
788    https://keskustelut.inderes.fi/t/investor-ab-w...
Name: URL, Length: 789, dtype: object


In [4]:
# Remove duplicate rows based on the 'URL' column
data_df_unique = data_df.drop_duplicates(subset=['URL'], keep=False)

# Check the shape of the DataFrame after removing duplicates
print("Shape of DataFrame after removing duplicates:", data_df_unique.shape)

Shape of DataFrame after removing duplicates: (789, 8)


In [5]:
# Define a custom function to extract details or return None for missing values
def extract_details(x, key):
    """
    Extract details from a dictionary or return None if the key is missing.

    Parameters:
        x (dict): The dictionary containing the details.
        key (str): The key to extract from the dictionary.

    Returns:
        Any: The value corresponding to the key if it exists, otherwise None.
    """
    if isinstance(x, dict):
        return x.get(key)
    else:
        return None
# Explode the 'Comments Details' column to create a new row for each comment
expanded_df = data_df.explode('Comments Details')

# Use apply to directly extract details or return None for missing values
expanded_df['Comment'] = expanded_df['Comments Details'].apply(lambda x: extract_details(x, 'comment'))
expanded_df['Timestamp'] = expanded_df['Comments Details'].apply(lambda x: extract_details(x, 'timestamp'))
expanded_df['Likes'] = expanded_df['Comments Details'].apply(lambda x: extract_details(x, 'likes'))
expanded_df['post_id'] = expanded_df['Comments Details'].apply(lambda x: extract_details(x, 'post_id'))

# Define a function to aggregate comments and merge with unique dataframe
def aggregate_comments(df):
    """
    Aggregate comments and merge with a unique dataframe.

    Parameters:
        df (pd.DataFrame): The DataFrame containing comments.

    Returns:
        pd.DataFrame: The DataFrame with aggregated comments merged with a unique dataframe.
    """
    # Create a subset for aggregation
    comment_subset = df[['URL', 'post_id', 'Comment']]

    # Aggregate comments
    aggregated_comments = comment_subset.groupby(['URL', 'post_id'])['Comment'].agg(' '.join).reset_index()

    # Remove duplicates from the original dataframe
    unique_df = df.drop_duplicates(subset=['URL', 'post_id']).drop('Comment', axis=1)

    # Merge the unique dataframe with the aggregated comments
    optimized_df = unique_df.merge(aggregated_comments, on=['URL', 'post_id'], how='inner')

    # Create Comment ID
    optimized_df['Comment ID'] = optimized_df['URL'] + '_' + optimized_df['post_id']

    return optimized_df.drop(['URL', 'post_id'], axis=1)
# Call the function to aggregate comments and merge with unique dataframe
optimized_df = aggregate_comments(expanded_df)

# Reset index
optimized_df.reset_index(drop=True, inplace=True)

# Drop the "Comments Details" column
optimized_df.drop(columns=['Comments Details'], inplace=True)

# Display the first few rows
print(optimized_df.head())


   Created At  Last Reply  Visits Replies Users Post Likes  \
0  touko 2018  joulu 2023  19,0 k      71    48        545   
1  touko 2018  joulu 2023  19,0 k      71    48        545   
2  touko 2018  joulu 2023  19,0 k      71    48        545   
3  touko 2018  joulu 2023  19,0 k      71    48        545   
4  touko 2018  joulu 2023  19,0 k      71    48        545   

                   Timestamp  Likes  \
0  18. toukokuuta 2018 11.48    0.0   
1  18. toukokuuta 2018 11.50    0.0   
2  18. toukokuuta 2018 11.55    0.0   
3  18. toukokuuta 2018 12.59    0.0   
4   20. toukokuuta 2018 5.13    0.0   

                                             Comment  \
0  Tässä ketjussa voi kehua omia onnistumisiaan v...   
1  Ostin Comptelia ja Nokia osti sen pois seuraav...   
2  Mä holdasin Comptelia kauan ennen kuin Nokia o...   
3  Nokia aiheeseen liittyen… Ostin Nokiaa juuri e...   
4  No Neste 2017 ylivoimainen oston sattuessa hal...   

                                          Comment ID  


In [6]:

optimized_df.tail()

,Created At,Last Reply,Visits,Replies,Users,Post Likes,Timestamp,Likes,Comment,Comment ID
107408,huhti 2020,22. maalis,"68,2 k",221,58,"1,4 k",20. lokakuuta 2023 21.22,0.0,Jatkaa kuitenkin Investorin omistamien yhtiöid...,https://keskustelut.inderes.fi/t/investor-ab-w...
107409,huhti 2020,22. maalis,"68,2 k",221,58,"1,4 k",25. marraskuuta 2023 1.35,0.0,"Juttu Atlas Copcoon, Ericssoniin ja Electrolux...",https://keskustelut.inderes.fi/t/investor-ab-w...
107410,huhti 2020,22. maalis,"68,2 k",221,58,"1,4 k",19. tammikuuta 2024 10.55,0.0,"2024-01-19 08:15 GMT+01 “At the end of 2023, ...",https://keskustelut.inderes.fi/t/investor-ab-w...
107411,huhti 2020,22. maalis,"68,2 k",221,58,"1,4 k",19. tammikuuta 2024 11.27,0.0,This was the thirteenth straight year of Inves...,https://keskustelut.inderes.fi/t/investor-ab-w...
107412,huhti 2020,22. maalis,"68,2 k",221,58,"1,4 k",22. maaliskuuta 2024 11.07,0.0,Investorin substanssialennus on hieman pienent...,https://keskustelut.inderes.fi/t/investor-ab-w...


In [7]:
df = optimized_df.copy()

In [8]:
pd.set_option('display.max_colwidth', None)  # Shows full content in columns
pd.set_option('display.max_rows', 100)       # Show up to 100 rows (adjust as needed)
pd.set_option('display.max_columns', None) 

In [9]:
# Identify duplicate timestamps
duplicate_timestamps = df[df.duplicated(subset=['Timestamp'], keep=False)]

# Display two rows with the same timestamp
if not duplicate_timestamps.empty:
    print("Two rows with the same timestamp:")
    print(duplicate_timestamps.head(20))
else:
    print("No rows with the same timestamp found.")

Two rows with the same timestamp:
      Created At   Last Reply  Visits Replies Users Post Likes  \
34    touko 2018   joulu 2023  19,0 k      71    48        545   
38    touko 2018   joulu 2023  19,0 k      71    48        545   
45    touko 2018   joulu 2023  19,0 k      71    48        545   
53    touko 2018   joulu 2023  19,0 k      71    48        545   
58    touko 2018   joulu 2023  19,0 k      71    48        545   
99   maalis 2020    4. maalis  21,3 k     139    24        457   
101  maalis 2020    4. maalis  21,3 k     139    24        457   
102  maalis 2020    4. maalis  21,3 k     139    24        457   
132  maalis 2020    4. maalis  21,3 k     139    24        457   
155  maalis 2020    4. maalis  21,3 k     139    24        457   
174  maalis 2020    4. maalis  21,3 k     139    24        457   
220   joulu 2020  maalis 2021  15,4 k     138    35        546   
223   joulu 2020  maalis 2021  15,4 k     138    35        546   
224   joulu 2020  maalis 2021  15,4 k     

In [10]:
# Check for duplicate timestamps
duplicate_timestamps = df[df.duplicated(subset=['Timestamp'], keep=False)]

# Print the duplicate timestamps
print(duplicate_timestamps)

        Created At  Last Reply  Visits Replies Users Post Likes  \
34      touko 2018  joulu 2023  19,0 k      71    48        545   
38      touko 2018  joulu 2023  19,0 k      71    48        545   
45      touko 2018  joulu 2023  19,0 k      71    48        545   
53      touko 2018  joulu 2023  19,0 k      71    48        545   
58      touko 2018  joulu 2023  19,0 k      71    48        545   
...            ...         ...     ...     ...   ...        ...   
107340  huhti 2020  22. maalis  68,2 k     221    58      1,4 k   
107351  huhti 2020  22. maalis  68,2 k     221    58      1,4 k   
107392  huhti 2020  22. maalis  68,2 k     221    58      1,4 k   
107396  huhti 2020  22. maalis  68,2 k     221    58      1,4 k   
107397  huhti 2020  22. maalis  68,2 k     221    58      1,4 k   

                         Timestamp  Likes  \
34      19. maaliskuuta 2021 22.27    0.0   
38       22. maaliskuuta 2021 9.13    0.0   
45        23. huhtikuuta 2021 9.54    0.0   
53       21. he

In [11]:

# Exclude 'Links' from the columns checked for nulls
columns_to_check = optimized_df.columns.difference(['Links'])
optimized_df = optimized_df.dropna(subset=columns_to_check)

In [12]:
optimized_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106644 entries, 0 to 107412
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Created At  106644 non-null  object 
 1   Last Reply  106644 non-null  object 
 2   Visits      106644 non-null  object 
 3   Replies     106644 non-null  object 
 4   Users       106644 non-null  object 
 5   Post Likes  106644 non-null  object 
 6   Timestamp   106644 non-null  object 
 7   Likes       106644 non-null  float64
 8   Comment     106644 non-null  object 
 9   Comment ID  106644 non-null  object 
dtypes: float64(1), object(9)
memory usage: 8.9+ MB


In [13]:
df = optimized_df.copy()

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106644 entries, 0 to 107412
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Created At  106644 non-null  object 
 1   Last Reply  106644 non-null  object 
 2   Visits      106644 non-null  object 
 3   Replies     106644 non-null  object 
 4   Users       106644 non-null  object 
 5   Post Likes  106644 non-null  object 
 6   Timestamp   106644 non-null  object 
 7   Likes       106644 non-null  float64
 8   Comment     106644 non-null  object 
 9   Comment ID  106644 non-null  object 
dtypes: float64(1), object(9)
memory usage: 8.9+ MB


In [15]:
# Define regular expression patterns for company mentions
all_company_patterns = {
    "Neste Oyj": r"(?i)\bneste\w*",
    "KONE Oyj": r"(?i)\bkone\w*",
    "Sampo Oyj": r"(?i)\bsampo\w*",
    "UPM-Kymmene Oyj": r"(?i)\bupm\w*",
    "Nokia Corporation": r"(?i)\bnokia\w*",
    "Fortum Oyj": r"(?i)\bfortum\w*",
    "Stora Enso Oyj": r"(?i)\bstora\w*enso\w*",
    "Wärtsilä Oyj Abp": r"(?i)\bwärtsilä\w*",
    "Metso Outotec Oyj": r"(?i)\bmetso\w*",
    "Kesko Oyj": r"(?i)\bkesko\w*",
    "Elisa Oyj": r"(?i)\belisa\w*",
    "Orion Oyj": r"(?i)\borion\w*",
    "Valmet Oyj": r"(?i)\bvalmet\w*",
    "Huhtamäki Oyj": r"(?i)\bhuhtamäki\w*",
    "Cargotec Corp": r"(?i)\bcargotec\w*",
    "Konecranes Plc": r"(?i)\bkonecranes\w*",
    "Kojamo Oyj": r"(?i)\bkojamo\w*",
    "Metsä Board Oyj": r"(?i)\bmetsä\w*",
    "Kemira Oyj": r"(?i)\bkemira\w*",
    "TietoEVRY Oyj": r"(?i)\btieto\w*",
    "Uponor Oyj": r"(?i)\buponor\w*",
    "Outokumpu Oyj": r"(?i)\boutokumpu\w*",
    "QT Group Oyj": r"(?i)\bqt\w*",
    "Fiskars OYJ Abp": r"(?i)\bfiskars\w*",
    "Vaisala Oyj": r"(?i)\bvaisala\w*",
    "Caverion Oyj": r"(?i)\bcaverion\w*",
    "Nokian Renkaat Oyj": r"(?i)\bnokian\w*",
    "Sanoma Oyj": r"(?i)\bsanoma\w*",
    "Enento Group Oyj": r"(?i)\benento\w*",
    "Terveystalo Oyj": r"(?i)\bterveystalo\w*",
    "Citycon Oyj": r"(?i)\bcitycon\w*",
    "Tokmanni Group Oyj": r"(?i)\btokmanni\w*",
    "Musti Group Oyj": r"(?i)\bmusti\w*",
    "Alma Media Oyj": r"(?i)\balma\w*",
    "Finnair Oyj": r"(?i)\bfinnair\w*",
    "Oma Säästöpankki Oyj": r"(?i)\boma\w*säästöpankki\w*",
    "Revenio Group Oyj": r"(?i)\brevenio\w*",
    "Ponsse Oyj": r"(?i)\bponsse\w*",
    "eQ Oyj": r"(?i)\beq\w*",
    "Olvi Oyj": r"(?i)\bolvi\w*",
    "Evli Pankki Oyj": r"(?i)\bevli\w*",
    "Marimekko Oyj": r"(?i)\bmarimekko\w*",
    "Harvia Oyj": r"(?i)\bharvia\w*",
    "Scanfil Oyj": r"(?i)\bscanfil\w*",
    "Ålandsbanken Abp": r"(?i)\bålandsbanken\w*",
    "Lassila & Tikanoja Oyj": r"(?i)\blassila\w*",
    "CapMan Oyj": r"(?i)\bcapman\w*",
    "Etteplan Oyj": r"(?i)\betteplan\w*",
    "Gofore Oyj": r"(?i)\bgofore\w*",
    "Viking Line ABP": r"(?i)\bviking\w*line\w*",
    "Raisio plc": r"(?i)\braisio\w*",
    "Talenom Oyj": r"(?i)\btalenom\w*",
    "Atria Oyj": r"(?i)\batria\w*",
    "Relais Group Oyj": r"(?i)\brelais\w*",
    "Incap Oyj": r"(?i)\bincap\w*",
    "Admicom Oyj": r"(?i)\badmicom\w*",
    "Kamux Oyj": r"(?i)\bkamux\w*",
    "Detection Technology Oyj": r"(?i)\bdetection\w*technology\w*",
    "Oriola Oyj": r"(?i)\boriola\w*",
    "Aspo Oyj": r"(?i)\baspo\w*",
    "NoHo Partners Oyj": r"(?i)\bnoho\w*",
    "Titanium Oyj": r"(?i)\btitanium\w*",
    "Bittium Oyj": r"(?i)\bbittium\w*",
    "Suominen Oyj": r"(?i)\bsuominen\w*",
    "Pihlajalinna Oyj": r"(?i)\bpihlajalinna\w*",
    "Verkkokauppa.com Oyj": r"(?i)\bverkkokauppa\w*",
    "Sitowise Group Oyj": r"(?i)\bsitowise\w*",
    "Afarak Group Oyj": r"(?i)\bafarak\w*",
    "Tecnotree Oyj": r"(?i)\btecnotree\w*",
    "Keskisuomalainen Oyj": r"(?i)\bkeskisuomalainen\w*",
    "Nurminen Logistics Oyj": r"(?i)\bnurminen\w*",
    "Consti Oyj": r"(?i)\bconsti\w*"
}

# Initialize the NER (Named Entity Recognition) model
model_checkpoint = "Kansallisarkisto/finbert-ner"
ner_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="simple")

# Pre-compile regular expressions for efficient pattern matching
compiled_patterns = {company_name: re.compile(pattern, re.IGNORECASE) for company_name, pattern in all_company_patterns.items()}

def process_comments(comment, compiled_patterns, ner_classifier):
    """
    Process comments to extract mentions of companies using pre-defined patterns and NER classification.

    Parameters:
        comment (str): The comment to process.
        compiled_patterns (dict): A dictionary containing pre-compiled regular expression patterns for company mentions.
        ner_classifier: The NER classifier model used to identify organization names.

    Returns:
        list: A list of tuples containing validated matches of company mentions in the comment.
              Each tuple contains the matched text, company name, and a boolean indicating if it's an organization.
    """
    validated_matches = []
    
    # Loop through precompiled patterns and search for matches
    for company_name, pattern in compiled_patterns.items():
        match = pattern.search(comment)
        if match:
            match_text = match.group()
            
            # Use NER classifier to check if the match is an organization name
            ner_results = ner_classifier(match_text)
            is_org = any(entity_info['entity_group'] == 'ORG' for entity_info in ner_results)
            
            # Append the validated match to the list
            if is_org:
                validated_matches.append((match_text, company_name, is_org))
    
    # Print the validated matches after processing the entire comment
    if validated_matches:
        print("Processed comment:", validated_matches)
    
    return validated_matches

# Apply processing to each comment in the DataFrame
print("Processing comments in DataFrame...")
df['Matched Companies Info'] = df['Comment'].apply(lambda x: process_comments(x, compiled_patterns, ner_classifier))
print("Processing complete.")

Processing comments in DataFrame...
Processed comment: [('Nokia', 'Nokia Corporation', True)]
Processed comment: [('Nokia', 'Nokia Corporation', True)]
Processed comment: [('Nokia', 'Nokia Corporation', True)]
Processed comment: [('Neste', 'Neste Oyj', True)]
Processed comment: [('Nokia', 'Nokia Corporation', True), ('Reveniossa', 'Revenio Group Oyj', True)]
Processed comment: [('Sampoa', 'Sampo Oyj', True)]
Processed comment: [('nokia', 'Nokia Corporation', True)]
Processed comment: [('Talenomiin', 'Talenom Oyj', True)]
Processed comment: [('NoHon', 'NoHo Partners Oyj', True)]
Processed comment: [('Harvian', 'Harvia Oyj', True)]
Processed comment: [('Nokiaa', 'Nokia Corporation', True)]
Processed comment: [('Nokiaa', 'Nokia Corporation', True), ('Wärtsilä', 'Wärtsilä Oyj Abp', True), ('Fiskarssia', 'Fiskars OYJ Abp', True), ('Nokian', 'Nokian Renkaat Oyj', True), ('Revenio', 'Revenio Group Oyj', True)]
Processed comment: [('Revenio', 'Revenio Group Oyj', True)]
Processed comment: [('R

In [16]:
data = df.copy()
data.tail()

,Created At,Last Reply,Visits,Replies,Users,Post Likes,Timestamp,Likes,Comment,Comment ID,Matched Companies Info
107408,huhti 2020,22. maalis,"68,2 k",221,58,"1,4 k",20. lokakuuta 2023 21.22,0.0,"Jatkaa kuitenkin Investorin omistamien yhtiöiden hallituksissa istumista (jatkossa täysipäiväisesti, kun ei CEO:n hommat häiritse). Ei ole siis sinänsä lähdössä minnekään, toimenkuva vain vaihtuu. Osaava ja aikaansaava kaveri, joten hyvä, että pysyy kuvioissa. Investorin pitkän aikavälin kehitykselle tärkeintä on omistusten liiketoiminnan kehittyminen. Näihin liiketoimintoihin ei paljoa vaikuta se, että kuka istuu toimarin pallilla. Itse näen tämän pudotuksen lisäyspaikkana ja heitinkin jo verkot vesille. Katsotaan jos tarttuisi jokunen lappu. Kuulisin kyllä mielellään eriäviä mielipiteitä. En ole mitenkään mahdottomasti eri mieltä, mutta onhan tuo silti negatiivinen uutinen, että lähtee CEO, jonka aikana yhtiöllä on mennyt noin pirun hyvin. Pienoinen reaktio alaspäin on varmaan ihan perusteltua. Jos sattuisi laskemaan enemmän niin ihan varmasti olis lisäyksen paikka, mutta tämä -4% ei aiheuta toimenpiteitä.",https://keskustelut.inderes.fi/t/investor-ab-wallenbergien-kruununjalokivi_post_234,[]
107409,huhti 2020,22. maalis,"68,2 k",221,58,"1,4 k",25. marraskuuta 2023 1.35,0.0,"Juttu Atlas Copcoon, Ericssoniin ja Electroluxiin liittyen. Kahden suuren Wallenberg-yhtiön toimitusjohtajat vaihtuvat ja kahden toisen yhtiön toimitusjohtajille vaaditaan potkuja.",https://keskustelut.inderes.fi/t/investor-ab-wallenbergien-kruununjalokivi_post_235,[]
107410,huhti 2020,22. maalis,"68,2 k",221,58,"1,4 k",19. tammikuuta 2024 10.55,0.0,"2024-01-19 08:15 GMT+01 “At the end of 2023, our adjusted net asset value reached an all-time-high, passing the SEK 800bn mark. During the year, adjusted net asset value growth and total shareholder return were 24 and 26 percent respectively, while the SIXRX return index gained 19 percent. This was the thirteenth straight year of Investor outperforming the stock market. In today’s complex environment, our focus remains on future-proofing our companies through strategic initiatives, while never compromising on efficiency.” Johan Forssell\nPresident & CEO of Investor",https://keskustelut.inderes.fi/t/investor-ab-wallenbergien-kruununjalokivi_post_236,[]
107411,huhti 2020,22. maalis,"68,2 k",221,58,"1,4 k",19. tammikuuta 2024 11.27,0.0,This was the thirteenth straight year of Investor outperforming the stock market. Huikea tilasto. On se vaan kova.,https://keskustelut.inderes.fi/t/investor-ab-wallenbergien-kruununjalokivi_post_237,[]
107412,huhti 2020,22. maalis,"68,2 k",221,58,"1,4 k",22. maaliskuuta 2024 11.07,0.0,"Investorin substanssialennus on hieman pienentynyt alkuvuoden aikana, ollen nyt 7% (ibindex). Jos katsoin oikein, niin Investor on kasvattanut kirja-arvoa per osake nopeammin kuin Latour 10v ja kaikilla sitä lyhyemmillä aikajänteillä. Latour arvostetaan 37% substanssipreemiolla. Preemiolle on varmasti perusteluita. Kuitenkin arvostusero Investoriin on melkoinen, jos vertaa yhtiöiden 10 vuoden suorituksia. Mielenkiintoista nähdä, onko arvostusero kutistumaan päin ja jos on, niin kumpi osakkeista lähentyy toisen arvostusta.",https://keskustelut.inderes.fi/t/investor-ab-wallenbergien-kruununjalokivi_post_240,[]


In [17]:
# Filter out rows with empty "Matched Companies Info"
data = data[data['Matched Companies Info'].apply(lambda x: len(x) > 0)]

# Reset the index after filtering
data.reset_index(drop=True, inplace=True)

data = data.drop(columns=['Last Reply'])

In [18]:
data.head(20)

,Created At,Visits,Replies,Users,Post Likes,Timestamp,Likes,Comment,Comment ID,Matched Companies Info
0,touko 2018,"19,0 k",71,48,545,18. toukokuuta 2018 11.50,0.0,Ostin Comptelia ja Nokia osti sen pois seuraavana päivänä,https://keskustelut.inderes.fi/t/onnistumiset-sijoittamisessa_post_2,"[(Nokia, Nokia Corporation, True)]"
1,touko 2018,"19,0 k",71,48,545,18. toukokuuta 2018 11.55,0.0,Mä holdasin Comptelia kauan ennen kuin Nokia osti sen.,https://keskustelut.inderes.fi/t/onnistumiset-sijoittamisessa_post_3,"[(Nokia, Nokia Corporation, True)]"
2,touko 2018,"19,0 k",71,48,545,18. toukokuuta 2018 12.59,0.0,Nokia aiheeseen liittyen… Ostin Nokiaa juuri ennen kuin Inderes lisäsi sen mallisalkkuunsa,https://keskustelut.inderes.fi/t/onnistumiset-sijoittamisessa_post_4,"[(Nokia, Nokia Corporation, True)]"
3,touko 2018,"19,0 k",71,48,545,20. toukokuuta 2018 5.13,0.0,No Neste 2017 ylivoimainen oston sattuessa halpaan hintaan.,https://keskustelut.inderes.fi/t/onnistumiset-sijoittamisessa_post_5,"[(Neste, Neste Oyj, True)]"
4,touko 2018,"19,0 k",71,48,545,20. toukokuuta 2018 14.26,0.0,Mä oon ollut Reveniossa pitkään ja Nokia alkuvuodesta! Kiitos siitä !,https://keskustelut.inderes.fi/t/onnistumiset-sijoittamisessa_post_6,"[(Nokia, Nokia Corporation, True), (Reveniossa, Revenio Group Oyj, True)]"
5,touko 2018,"19,0 k",71,48,545,12. syyskuuta 2019 15.25,0.0,"Huomasin juuri salkun olevan ATH-lukemissa. Metsämaa ja osakkeet kun lasketaan yhteen niin ollaan jossain 240 000 euron kulmilla. Asuntoa en laske mukaan. Melko hyvin keskituloiselle duunarille, vaikka itse sanonkin. Kaikki taisi alkaa 2004 elokuussa, kun täytettyäni 18 mietin mihin laitan kesätöistä säästämäni rahat. No siitä marssimaan osuuspankkiin ja tili auki. Sampoa ostin ensimmäisenä ja vieläkin omistan osan näistä. Toisena tarttui Lemminkäistä, mutta niistä luovuin parin vuoden jälkeen. Palkasta säästämällä ja nollista lähtemällä tuo on kieltämättä melko haasteellista saavuttaa näillä tuloilla. Vipua on tullut käytettyä välillä hyvinkin paljon, mutta se on toiminut oikeaan suuntaan. Nykyään aika maltillisesti verrattuna salkun kokoon Yritetään jatkaa samaan malliin. Salkku ja metsät alkavat kyllä hiljalleen olla cruise-modessa. Vähän raivaussahaa ja osta/myy -painiketta saa aika ajoin riittää",https://keskustelut.inderes.fi/t/onnistumiset-sijoittamisessa_post_8,"[(Sampoa, Sampo Oyj, True)]"
6,touko 2018,"19,0 k",71,48,545,29. syyskuuta 2019 20.40,0.0,"Kyllä mun mielestä lapsen salkussa, jossa summat ei ole vielä korkeita, voi mennä keskittämällä korkeintaan muutamiin tuotto-odotukseltaan / profiileiltaan parhaisiin keisseihin. Näin olen ajatellut viimeisimmät vuodet. Meilläkin oli pitkään vain 2-3 lappua kerrallaan ja vasta tänä syksynä lukumäärää nostatin tuohon viiteen… @Astrix sun taktiikka on mun mielestäni just hyvä Ulkomuistista tarina menee jotenkin näin: 2016-2017 alkupäässä mulla oli lähinnä supereita, koska taktiikkana on suht tasainen kuukausisäästäminen… sitten ehkä vajaan vuoden jälkeen tein taktiikan muutoksen, joka toistaiseksi on näyttänyt kannattavan. (Huom nykyinen 3v periodi on vielä kuitenkin lyhyt.) 2017-2018 Ensimmäiset suorat ostot meni tuolla hoivatilat-remedy painotuksella, jossa sivussa myös yksi onnistunut nokia-veivi (osto 3.95 → myynti 5.3€). Remedyä olin ostanut 7€ ja 6.8€ sekä Hoivatiloja 8€:llä. 2019 H1: Säästämisen edetessä mukaan tuli myös kamux (kevät-kesä 2019?). Tuo onkin ainut vielä turskalla oleva (<5% turskaa nyt), mutta eiköhän tuo 15 vuoden sisällä nouse vielä plussalle 2019 H2: Remedyn pahimman aliarvostuksen purkaannuttua ja ensimmäisten gamstat-laskelmien jälkeen kevensin lappua (myynti 10.65€) silloisesta noin 50% painosta nykyiselle tasolleen. (Samalla hetkellä, kuin omastakin salkusta kevensin). Tuosta vapautuneilla varoilla sitten tilalle QT groupia (13.6€) ja Nordeaa (5.8€). (Myös omaan salkkuun livahti samalla hetkellä Nordeaa) Nykyisistä positioista ajattelin roikottaa pitkäänkin lähes kaikkia, mutta Nordean osalta odottelen C

In [20]:
import datetime
# Get the last modified time as a timestamp
timestamp = os.path.getmtime(file_path1)

# Convert the timestamp to a readable datetime
last_modified_time = datetime.datetime.fromtimestamp(timestamp)

print(f"The file '{file_path1}' was last modified on: {last_modified_time}")

The file 'forum_data_scraped2' was last modified on: 2024-04-22 03:38:06.157366


In [25]:
# Adjust the current_date if your context is not today
context_date = datetime.datetime.fromtimestamp(timestamp)

# Mapping for Finnish month abbreviations
month_mapping = {
    'tammi': '01', 'helmi': '02', 'maalis': '03', 'huhti': '04', 'touko': '05',
    'kesä': '06', 'heinä': '07', 'elo': '08', 'syys': '09', 'loka': '10',
    'marras': '11', 'joulu': '12',
    'tammikuuta': '01', 'helmikuuta': '02', 'maaliskuuta': '03', 'huhtikuuta': '04',
    'toukokuuta': '05', 'kesäkuuta': '06', 'heinäkuuta': '07', 'elokuuta': '08',
    'syyskuuta': '09', 'lokakuuta': '10', 'marraskuuta': '11', 'joulukuuta': '12'
}

# Function to convert Finnish date to standard format (with four-digit year)
def parse_created_at(date_str):
    parts = date_str.split(' ')
    if len(parts) == 2:
        month = parts[0].strip().lower()
        year = parts[1].strip()
        if month in month_mapping:
            return f"{year}-{month_mapping[month]}-01"
    return None

# Function to parse full timestamps like "25. toukokuuta 2022 5.49"
def parse_timestamp(date_str):
    parts = date_str.split(' ')
    if len(parts) == 4:
        day = parts[0].strip().rstrip('.')
        month = parts[1].strip().lower()
        year = parts[2].strip()
        time = parts[3].replace('.', ':')
        if month in month_mapping:
            return f"{year}-{month_mapping[month]}-{day.zfill(2)} {time}"
    return None

# Apply the parsing functions to your DataFrame columns
data['Created At'] = data['Created At'].apply(parse_created_at)
data['Timestamp'] = data['Timestamp'].apply(parse_timestamp)

print(data)

       Created At  Visits Replies Users Post Likes         Timestamp  Likes  \
0      2018-05-01  19,0 k      71    48        545  2018-05-18 11:50    0.0   
1      2018-05-01  19,0 k      71    48        545  2018-05-18 11:55    0.0   
2      2018-05-01  19,0 k      71    48        545  2018-05-18 12:59    0.0   
3      2018-05-01  19,0 k      71    48        545   2018-05-20 5:13    0.0   
4      2018-05-01  19,0 k      71    48        545  2018-05-20 14:26    0.0   
...           ...     ...     ...   ...        ...               ...    ...   
10160  2020-04-01  68,2 k     221    58      1,4 k   2022-05-25 5:49    0.0   
10161  2020-04-01  68,2 k     221    58      1,4 k  2022-08-26 17:40    0.0   
10162  2020-04-01  68,2 k     221    58      1,4 k  2022-08-26 17:55    0.0   
10163  2020-04-01  68,2 k     221    58      1,4 k  2022-11-08 10:07    0.0   
10164  2020-04-01  68,2 k     221    58      1,4 k  2022-11-18 17:28    0.0   

                                                   

In [26]:
data.info()
df = data.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10165 entries, 0 to 10164
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Created At              10147 non-null  object 
 1   Visits                  10165 non-null  object 
 2   Replies                 10165 non-null  object 
 3   Users                   10165 non-null  object 
 4   Post Likes              10165 non-null  object 
 5   Timestamp               10165 non-null  object 
 6   Likes                   10165 non-null  float64
 7   Comment                 10165 non-null  object 
 8   Comment ID              10165 non-null  object 
 9   Matched Companies Info  10165 non-null  object 
dtypes: float64(1), object(9)
memory usage: 794.3+ KB


In [27]:
# Updated function to convert mixed numeric representations to integers
def convert_to_int(value_str):
    # Clean the string to remove any extra spaces or commas
    clean_value = value_str.strip().replace(',', '').lower()
    
    # Check for "k" (indicating thousands)
    if 'k' in clean_value:
        # Convert to float after removing "k", then multiply by 1,00
        return int(float(clean_value.replace('k', '')) * 100)
    
    # Check for "M" (indicating millions)
    elif 'm' in clean_value:
        # Convert to float after removing "M", then multiply by 1,000,000
        return int(float(clean_value.replace('m', '')) * 1_000_000)
    
    # Otherwise, convert directly to float and then to int
    return int(float(clean_value))

# Columns to apply the conversion
columns_to_convert = ['Visits', 'Replies', 'Users', 'Post Likes', 'Likes']

# Applying the conversion function to the specified columns
for column in columns_to_convert:
    df[column] = df[column].astype(str).apply(convert_to_int)
# Converting time-related columns to datetime
df['Created At'] = pd.to_datetime(df['Created At'], errors='coerce')  
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Display the updated DataFrame to confirm the changes
print(df.head())  # Adjust this to display the DataFrame as you prefer

  Created At  Visits  Replies  Users  Post Likes           Timestamp  Likes  \
0 2018-05-01   19000       71     48         545 2018-05-18 11:50:00      0   
1 2018-05-01   19000       71     48         545 2018-05-18 11:55:00      0   
2 2018-05-01   19000       71     48         545 2018-05-18 12:59:00      0   
3 2018-05-01   19000       71     48         545 2018-05-20 05:13:00      0   
4 2018-05-01   19000       71     48         545 2018-05-20 14:26:00      0   

                                                                                      Comment  \
0                                   Ostin Comptelia ja Nokia osti sen pois seuraavana päivänä   
1                                      Mä holdasin Comptelia kauan ennen kuin Nokia osti sen.   
2  Nokia aiheeseen liittyen… Ostin Nokiaa juuri ennen kuin Inderes lisäsi sen mallisalkkuunsa   
3                                 No Neste 2017 ylivoimainen oston sattuessa halpaan hintaan.   
4                      Mä oon ollut Reve

In [28]:
df.info()
df2 = df.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10165 entries, 0 to 10164
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Created At              10147 non-null  datetime64[ns]
 1   Visits                  10165 non-null  int64         
 2   Replies                 10165 non-null  int64         
 3   Users                   10165 non-null  int64         
 4   Post Likes              10165 non-null  int64         
 5   Timestamp               10165 non-null  datetime64[ns]
 6   Likes                   10165 non-null  int64         
 7   Comment                 10165 non-null  object        
 8   Comment ID              10165 non-null  object        
 9   Matched Companies Info  10165 non-null  object        
dtypes: datetime64[ns](2), int64(5), object(3)
memory usage: 794.3+ KB


In [29]:
# Define the regex pattern for URLs
url_pattern = re.compile(r'https?://\S+|www\.\S+')

# Preprocess the comment data
df2["Comment"] = df2["Comment"].apply(lambda text: re.sub(r"[^a-zA-Z0-9ÄÖÅäöå ]", "", text.lower()).strip())
df2["Comment"] = df2["Comment"].apply(lambda text: url_pattern.sub(r'', text))

# Now the "text" column has been renamed to "comment" and preprocessed
print(df2.head())

  Created At  Visits  Replies  Users  Post Likes           Timestamp  Likes  \
0 2018-05-01   19000       71     48         545 2018-05-18 11:50:00      0   
1 2018-05-01   19000       71     48         545 2018-05-18 11:55:00      0   
2 2018-05-01   19000       71     48         545 2018-05-18 12:59:00      0   
3 2018-05-01   19000       71     48         545 2018-05-20 05:13:00      0   
4 2018-05-01   19000       71     48         545 2018-05-20 14:26:00      0   

                                                                                     Comment  \
0                                  ostin comptelia ja nokia osti sen pois seuraavana päivänä   
1                                      mä holdasin comptelia kauan ennen kuin nokia osti sen   
2  nokia aiheeseen liittyen ostin nokiaa juuri ennen kuin inderes lisäsi sen mallisalkkuunsa   
3                                 no neste 2017 ylivoimainen oston sattuessa halpaan hintaan   
4                         mä oon ollut reveni

In [30]:
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10165 entries, 0 to 10164
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Created At              10147 non-null  datetime64[ns]
 1   Visits                  10165 non-null  int64         
 2   Replies                 10165 non-null  int64         
 3   Users                   10165 non-null  int64         
 4   Post Likes              10165 non-null  int64         
 5   Timestamp               10165 non-null  datetime64[ns]
 6   Likes                   10165 non-null  int64         
 7   Comment                 10165 non-null  object        
 8   Comment ID              10165 non-null  object        
 9   Matched Companies Info  10165 non-null  object        
dtypes: datetime64[ns](2), int64(5), object(3)
memory usage: 794.3+ KB
None


In [31]:
# Initialize the sentiment analysis pipeline with the finbert-finnsentiment model
sentiment_pipeline = pipeline("text-classification", model="fergusq/finbert-finnsentiment")

In [32]:
# Apply sentiment analysis to each comment and extract the sentiment label
def get_sentiment(text):
    # Truncate or split the text if it exceeds the maximum length
    max_length = 512
    if len(text) > max_length:
        text = text[:max_length]  # Truncate the text to the maximum length
    # Get the first result (since the pipeline returns a list of results)
    result = sentiment_pipeline(text)
    return result[0]['label']  # Get the label of the first result

# Add a new "Sentiment" column by applying the sentiment function to the "Comment" column
df2['Sentiment'] = df2['Comment'].apply(get_sentiment)

In [33]:
# Save DataFrame to a pickle file
df2.to_pickle('preprocessed_forum_data5.pkl')


In [34]:
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 10150.562512874603 seconds


In [35]:
df2.tail(30)

Created At  Visits  Replies  Users  Post Likes           Timestamp  \
10135 2019-05-01    5600       30     13          61 2019-05-24 21:02:00   
10136 2019-05-01    5600       30     13          61 2019-05-25 13:09:00   
10137 2019-05-01    5600       30     13          61 2019-05-25 14:10:00   
10138 2019-05-01    5600       30     13          61 2019-05-25 19:42:00   
10139 2019-05-01    5600       30     13          61 2019-05-25 20:41:00   
10140 2019-05-01    5600       30     13          61 2019-05-28 17:28:00   
10141 2022-08-01   15200       89     14         706 2022-08-14 13:13:00   
10142 2022-08-01   15200       89     14         706 2023-06-02 10:16:00   
10143 2021-01-01    8400       48     11         105 2021-01-24 11:15:00   
10144 2021-01-01    8400       48     11         105 2021-02-11 12:36:00   
10145 2022-11-01   28400      152     33         805 2024-02-05 20:13:00   
10146 2022-03-01   11600       17      5         225 2022-03-20 12:28:00   
10147 2022-03-01   11600       17      5         225 2023-08-20 09:59:00   
10148 2020-11-01   30200     1300    185       11800 2021-08-04 11:38:00   
10149 2020-11-01   30200     1300    185       11800 2021-08-05 22:47:00   
10150 2020-04-01   68200      221     58        1400 2020-04-23 07:31:00   
10151 2020-04-01   68200      221     58        1400 2020-04-23 16:02:00   
10152 2020-04-01   68200      221     58        1400 2021-05-01 12:38:00   
10153 2020-04-01   68200      221     58        1400 2021-09-17 13:05:00   
10154 2020-04-01   68200      221     58        1400 2021-10-12 18:27:00   
10155 2020-04-01   68200      221     58        1400 2021-10-12 20:03:00   
10156 2020-04-01   68200      221     58        1400 2022-05-08 10:08:00   
10157 2020-04-01   68200      221     58        1400 2022-05-08 19:29:00   
10158 2020-04-01   68200      221     58        1400 2022-05-09 22:14:00   
10159 2020-04-01   68200      221     58        1400 2022-05-09 23:19:00   
10160 2020-04-01   68200      221     58        1400 2022-05-25 05:49:00   
10161 2020-04-01   68200      221     58        1400 2022-08-26 17:40:00   
10162 2020-04-01   68200      221     58        1400 2022-08-26 17:55:00   
10163 2020-04-01   68200      221     58        1400 2022-11-08 10:07:00   
10164 2020-04-01   68200      221     58        1400 2022-11-18 17:28:00   

       Likes  \
10135      0   
10136      0   
10137      0   
10138      0   
10139      0   
10140      0   
10141      0   
10142      0   
10143      0   
10144      0   
10145      0   
10146      0   
10147      0   
10148      0   
10149      0   
10150      0   
10151      0   
10152      0   
10153      0   
10154      0   
10155      0   
10156      0   
10157      0   
10158      0   
10159      0   
10160      0   
10161      0   
10162      0   
10163      0   
10164      0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                